In [ ]:







def getEachSeedLocationsInPage(document, (lSeed, rSeed), kvDiff=200):
    lSeedLocations = [(i, i+len(lSeed)) for i in range(len(document)) if document.startswith(lSeed, i)]
    docLength = len(document)
    output = []
    for (ls, le) in lSeedLocations:
        subStr = document[le: min(le+200, docLength)]
        loc = subStr.find(rSeed)
        if loc==-1:
            continue
        rs = loc
        re = loc + len(rSeed)
        output.append((ls, le, rs, re))
    return output
    

def getSeedsPositions(pageContent, presentSeeds):
    output = []
    for seed in presentSeeds:
        output.append(getEachSeedLocationsInPage(pageContent, seed))
    return output





In [97]:
import os
import re
from os import listdir
from os.path import isfile, join

def getAllDirectoriesInLocation(loc):
    listOfSubDir = [loc+"/"+f for f in os.listdir(loc)]
    return listOfSubDir

#do train/test split
def trainTestSplit(files):
    sixtyPercent = int(len(files)*(0.6))
    if sixtyPercent<4:
        print("We won't be able to report accuracy of patterns")
        return (files, [])
    else:
        return (files[:sixtyPercent], files[sixtyPercent:])

#read all realtion sseds    
def readAllRelationSeeds(seedsLocation):
    relationSeeds = []
    with open(seedsLocation) as f:
        for line in f:
            relationSeed = re.split(r'\t+', line)
            if len(relationSeed)!=2:
                continue
            key = removeMultipleWhiteSpaces(relationSeed[0].strip())
            value = removeMultipleWhiteSpaces(relationSeed[1].strip())
            relationSeeds.append((key, value))
    return set(relationSeeds)
    

def removeMultipleWhiteSpaces(s):
    return ' '.join(s.split())

def getPageLocationAndSeed(d):
    output = []
    for pageDir in d:
        pageLocation = pageDir + "/page.html"
        seedLocation = pageDir + "/seed"
        seed = readAllRelationSeeds(seedLocation)
        output.append((pageLocation, seed))
    return output


#read the page from pageLocation
def readPage(pageLocation):
    htmlPageContent = ""
    with open(pageLocation, 'r') as myfile:
        htmlPageContent = myfile.read().strip()
    return htmlPageContent

def removeTerminatedAnd(s):
    return re.sub("&amp;", "&", s)



#document processing logic at this stage is about removing multiple whitespaces into single one 
def preprocessDocument(document):
    return removeTerminatedAnd(' '.join(document.split()))


#get right contexts for each cluster    
def getRightContexts(seedsLocations, htmlPageContent):
    output = []
    for (ls, le, rs, re) in seedsLocations:
        start = le+rs
        end = le+re
        rc = htmlPageContent[end:min(len(htmlPageContent), end+100)].strip()
        if len(rc)<=0:
            continue
        output.append(rc)
    return output



def getLeftContexts(seedsLocations, htmlPageContent):
    output = []
    for (start, end, x, y) in seedsLocations:
        lc = htmlPageContent[(max(0, start-100)):start].strip()
#         print("Seed is ")
#         print(seed)
#         print(str((htmlPageContent[start:end], htmlPageContent[end+x:end+y])))
#         print("Left Context ")
#         print(lc)
        if len(lc)<=0:
            continue
        lc = lc[::-1]
        output.append(lc)
    return output
    


def getMiddleContexts(seedsLocations, pageContent):
    output = []
    for (x, y, start, end) in seedsLocations:
#         print("seed1 is " + str(pageContent[x:y]))
#         print("seed2 is " + str(pageContent[y+start:y+end]))
#         print("full contnet is " + str(pageContent[x:y+end]))
        mid = pageContent[y:y+start].strip()
#         print("mid is " + mid)
#         print("mid is" + mid)
        if len(mid)<=0:
            continue
        output.append(mid)
    return output




def getLeftBetweenRightContexts(pageContent, seed):
    b = getEachSeedLocationsInPage(pageContent, seed)
    output = []
    leftContexts    = getLeftContexts(b, pageContent)
    betweenContexts = getMiddleContexts(b, pageContent)
    rightContexts   = getRightContexts(b, pageContent)
#     print("Seed is " + str(seed))
#     print("right is " + str(rightContexts))
    return (leftContexts, betweenContexts, rightContexts)



def getContextsForAllSeeds(pageContent, seed):
    lout = []
    bout = []
    rout = []
    for s in seed:
        (lcPerS, bcPerS, rcPerS) = getLeftBetweenRightContexts(pageContent, s)
#         if len(lcPerS)==0:
#             print("Zero\n\n\n\n\n")
#             print("Seed is " + str(s))
        lout.append(lcPerS)
        bout.append(bcPerS)
        rout.append(rcPerS)
#         print("seed iss " + str(s))
#         print("ists patterns are " + str(lcPerS))
    return (lout, bout, rout)


def printContextInformation(contexts, pageAndSeed, rev=False):
    for index in range(0, len(pageAndSeed)):
        print("\n\n\n\n")
        (pageLocation, seed) = pageAndSeed[index]
        pageContent = preprocessDocument(readPage(pageLocation))
        contextPerPage = contexts[index]
        print("For seed " + str(seed))
        print("Context is following: " + pageLocation)# + " number " + str(contextPerPage))
        for item in contextPerPage:
            if rev==True:
                item = item[::-1]
            print(item)
            print("---------------------")
            
def printPerPageContextInfo(pageContent, contextPerPage, rev=False):
    for item in contextPerPage:
        if rev==True:
            item = item[::-1]
        print(item)
        print("---------------------")
    

    
def getCommonPrefix(s1, s2):
    l = [s1, s2]
    return os.path.commonprefix(l)



def insertCommonPrefix(results, commonPrefix):
    if commonPrefix in results or len(commonPrefix)<=0:
        return results
    r = list(results)
    for item in r:
        if len(item)<len(commonPrefix) and commonPrefix.startswith(item):
            results.remove(item)
    r=list(results)
    for item in r:
        if len(item)>len(commonPrefix) and item.startswith(commonPrefix):
            return results
    results.add(commonPrefix)
    return results

def doPrefixIntersection(list1, list2):
    results = set()
    for item1 in list1:
        commonPrefix = ""
        for item2 in list2:
            c = getCommonPrefix(item1, item2)
            if len(c)>len(commonPrefix):
                commonPrefix = c
        results = insertCommonPrefix(results, commonPrefix)
    return results


def doIntersection(patterns):
    if len(patterns)<=0:
        return patterns
    result = patterns[0]
    for index in range(1, len(patterns)):
#         print("Input")
#         print(result)
#         print(patterns[index])
        result = doPrefixIntersection(result, patterns[index])
#         print(result)
    return result


def printIn(l):
    for item in l:
        print("==========================")
        for val in item:
            print(val)
            print("---------------------")

def getLeftPatterns(leftContexts):
#     print("Left full contexts are ")
#     print(printIn(leftContexts))
    lp = doIntersection(leftContexts)
    results = []
    for item in lp:
        results.append(item[::-1]) 
    return results


def getRightPatterns(rightContexts):
#     print("Right full contexts are ")
#     print(printIn(rightContexts))
    return list(doIntersection(rightContexts))

def getBetweenPatterns(betweenContexts):
    flat_list = [item for sublist in betweenContexts for item in sublist]
    return list(set(flat_list))
#     print("Type was " + str(type(betweenContexts[0][0])))
#     return list(doIntersection(betweenContexts))

def allPossibleTriples(lc, bc, rc):
    output = []
    for l in lc:
        for b in bc:
            for r in rc:
                output.append((l, b, r))
    return output
                
    
    

        
#remove elements which were actually tag
def isTag((a,b)):
    if a.find("<")!=-1 and a.find(">")!=-1 and b.find("<")!=-1 and b.find(">")!=-1:
        return True
    return False


#preprocess results before writing it to file
def preprocessResults(output):
    result = []
    for o in output:
        if isTag(o):
            continue
        result.append(o)
    return result
    
    
    
def isDifferent(ch):
    if ch>='a' and ch<='z':
        return False
    elif ch>='A' and ch<='Z':
        return False
    elif ch>='0' and ch<='9':
        return False
    else:
        return True
    
    
def findExactWordLoc(document, seed):
    output = []
    for m in re.finditer(re.escape(seed), document):
        start = m.start()
        end   = m.end()
        output.append((start, end))
#     print(output)
    for (s, e) in output:
        if s==0:
            l = ' '
            if e == len(document):
                r = ' '
            else:
                r = document[e]
        else:
            l = document[s-1]
            if e==len(document):
                r = ' '
            else:
                r = document[e]
#         print((s, e))
#         print(l)
#         print(r)
#         print()
#         print(str(l) + " " + str(isDifferent(l)))
#         print(sisDifferent(r))
        if isDifferent(l) and isDifferent(r):
#             print("l was " + l)
#             print("r was " + r)
            return s
    if len(output)==0:
        return -1
    else:
        (s, e) = output[0]
        return s
    
# def findExactWordLoc(document, seed):
#     output = []
#     for m in re.finditer(re.escape(seed), document):
#         start = m.start()
#         end   = m.end()
#         output.append((start, end))
#     print(output)
#     for (s, e) in output:
#         if s==0:
#             l = ' '
#             if e == len(document):
#                 r = ' '
#             else:
#                 r = document[end]
#         else:
#             l = document[s-1]
#             if e==len(document):
#                 r = ' '
#             else:
#                 r = document[end]
#         if isDifferent(l) and isDifferent(r):
#             return start
#     if len(output)==0:
#         return -1
#     else:
#         (s, e) = output[0]
#         return s    
# def getLoc(doc, seed):
#     start = 0
#     end = 0
#     for m in re.finditer(re.escape("\b"+seed+"\b"), doc):
#         start = m.start()
#         end = m.end()
#         return int(start)
#     print("String is " + doc[start:])
#     return start
    
def getEachSeedLocationsInPage(document, (lSeed, rSeed), kvDiff=200):
    lSeedLocations = [(i, i+len(lSeed)) for i in range(len(document)) if document.startswith(lSeed, i)]
    docLength = len(document)
    output = []
    for (ls, le) in lSeedLocations:
        subStr = document[le: min(le+200, docLength)]
        start = 0
        end = 0
        loc = findExactWordLoc(subStr, rSeed)
        if loc==-1:
            continue
        rs = loc
        re = loc + len(rSeed)
        output.append((ls, le, rs, re))
    return output










    

#Pattern is (l, m, r) and match them to htmlPageContent
def findRelationSetwrtPattern(htmlPageContent, (l, mid, r)):
    #for each start location of pattern find its end
    #for each end page find the pattern right
    #extract everything till that point
    #after extraction move one point above that pattern string
    results = []
    for m in re.finditer(re.escape(l), htmlPageContent):
        start = m.start()
        end = m.end()
        rightPage = htmlPageContent[end:]
        rightLoc  = rightPage.find(r)
        if rightLoc==-1:
            break
        substr = rightPage[:rightLoc]
#         print("substr is " + substr)
        mLoc = substr.find(mid)
#         print("mloc is " + str(mLoc))
        if mLoc==-1:
            continue
        e1 = substr[:mLoc].strip()
        e2 = substr[mLoc+len(mid):].strip()
#         print(e1)
#         print(e2)
        if len(e1)>1 and len(e2)>=1 and len(e1)<100 and len(e2)<500:
#             print("Appending " + str((e1, e2)))
            results.append((e1, e2))
    return set(results)



#input: all the pattern found and new html page
#output: extract set of elements from the page that are in our set(whose definition we know)
def extractSet(patterns, htmlPageContent):
    output = []
    for pattern in patterns:
        output.extend(findRelationSetwrtPattern(htmlPageContent, pattern))
    return output



def doProcessingWithoutClass(s, regExpIn, regExpOut):
    allManaged =  re.sub(regExpIn, regExpOut, s)
    removeQuoteIn = r'"[^"]*"'
    removeQuoteOut = r'""'
    return re.sub(removeQuoteIn, removeQuoteOut, allManaged)
#     i = "class[\s]*=[\s]*"
#     o       = "class="
#     s = re.sub(i, o, s)
#     i = "([^class])=\"[^\"]*\""
#     o = "\\1=\"\""
#     return re.sub(i, o, s)



def getLastWordWithoutEqualSign(leftContext):
    words = leftContext.strip().split(" ")
    totalWords = len(words)
    if len(words)<2:
        return ""
    else:
        if words[totalWords-1]=="=":
            return words[totalWords-2]
        else:
            return ""


def getSpecialWords(document, seedsSet):
    totalLength = len(document)
    specialWords = []
    for s in seedsSet:
        allSeedPositions = getEachSeedLocationsInPage(document, s)
        for (start, end, rOffsetStart, rOffsetEnd) in allSeedPositions:
            prevLoc = start-1
            nextLoc = end
            prevLoc2 = end+rOffsetStart-1
            nextLoc2 = end+rOffsetEnd+1
            if prevLoc>=0 and nextLoc<totalLength:
                prevQuot = document[prevLoc]
                nextQuot  = document[nextLoc]
#                 print(prevQuot + " " + nextQuot)
                if prevQuot=="\"" and nextQuot=="\"":
                    leftContext = document[max(0, prevLoc-100):prevLoc]
                    lastWord = getLastWordWithoutEqualSign(leftContext)
                    if len(lastWord)>0:
                        specialWords.append(lastWord)
            if prevLoc2>=0 and nextLoc2<totalLength:
                prevQuot = document[prevLoc2]
                nextQuot  = document[nextLoc2]
#                 print(prevQuot + " " + nextQuot)
                if prevQuot=="\"" and nextQuot=="\"":
                    leftContext = document[max(0, prevLoc2-100):prevLoc2]
                    lastWord = getLastWordWithoutEqualSign(leftContext)
                    if len(lastWord)>0:
                        specialWords.append(lastWord)

    return specialWords



def createRegExpFromWordStr(wordsStr):
#     wordsStr = getWordsStr(words)
#     print("wordsstr is " + wordsStr)
    return ("(" + wordsStr +")[\s]*=[\s]*\"([^\"]*)\"", "\\1=\\2")



def getWordsStr(words):
    out = words[0]
    if len(words)==1:
        return out
    else:
        for index in range(1, len(words)):
            out+="|"+words[index]
    print(out)
    return out


def getAllSpecialWordsAsString(pageLocationAndSeed):
    specialWords = []
    specialWords.append("class")
    for (pageLocation, seeds) in pageLocationAndSeed:
        pageContent = preprocessDocument(readPage(pageLocation))
        specialWords.extend(getSpecialWords(pageContent, seeds))
    specialWords = list(set(specialWords))
    return getWordsStr(specialWords)




def filterPatterns(pageLocationAndSeeds, patterns, applyRegExp = False, regExpIn="", regExpOut=""):
    output = []
    for (pageLocation, seeds) in pageLocationAndSeeds:
        pageContent = preprocessDocument(readPage(pageLocation))
        if applyRegExp==True:
            pageContent = doProcessingWithoutClass(pageContent, regExpIn, regExpOut)
        for pattern in patterns:
            resultsPerPage = findRelationSetwrtPattern(pageContent, pattern)
            common = list(resultsPerPage.intersection(seeds))
            if len(common)>0:
                output.append(pattern)
    return set(output)

def prefixStatus((l1, r1), (l2, r2)):
#     print(l1, r1)
#     print(l2, r2)
    if (l1.startswith(l2) or l1.endswith(l2)) and len(l1)>len(l2):
        if r1==r2:
            l1 = l1.split(" ")
            if l1[len(l1)-1]!=l2:
                return True
    return False

#     else:
#         if (r1.startswith(r2) or r2.startswith(r1)) and len(r1)>len(r2):
#             return True
#         else:
#             return False
def removeJunk(s):
    l = list(s)
    total = len(l)
    output = []
    for index in range(0, total):
        flag=True
        for j in range(0, total):
            if index==j:
                continue
            if prefixStatus(l[index], l[j]):
                print("This guy")
                print(l[j])
                flag=False
                break
        if flag==True:
            print("************Saving*******************")
            print(l[index])
            output.append(l[index])
        else:
            print("*************Removing*****************")
            print(l[index])
    print("returning")
    print(set(output))
    return set(output)
                


def getAllPatternsByPlainStringMatch(corpusLocation):
    directories             = getAllDirectoriesInLocation(corpusLocation)
    (trainDirs, testDirs)   = trainTestSplit(directories)
    pageLocationsAndSeed    = getPageLocationAndSeed(trainDirs)
    testPageLocationAndSeed = getPageLocationAndSeed(testDirs)
    leftContexts    = []
    betweenContexts = []
    rightContexts   = []
    for (pageLocation, seed) in pageLocationsAndSeed:
        pageContent = preprocessDocument(readPage(pageLocation))
        (lcPerPage, bcPerPage, rcPerPage) = getContextsForAllSeeds(pageContent, seed)
#         (leftContextsPerPage, rightContextsPerPage) = getContextsForAllSeeds(pageContent, seed)
        leftContexts.extend(lcPerPage)
        betweenContexts.extend(bcPerPage)
        rightContexts.extend(rcPerPage)
#         print("Page is " + str(pageLocation))
#         print("texts are ")
#         print(rcPerPage)
#     printContextInformation(betweenContexts, pageLocationsAndSeed)
    leftPatterns     = getLeftPatterns(leftContexts)
    betweenPatterns  = getBetweenPatterns(betweenContexts)
    rightPatterns    = getRightPatterns(rightContexts)
    print("Left Patterns")
    print(leftPatterns)
    print("Between")
    print(betweenPatterns)
    print("Right")
    print(rightPatterns)
    patterns         = allPossibleTriples(leftPatterns, betweenPatterns, rightPatterns)
    print(patterns)
    patterns = filterPatterns(pageLocationsAndSeed, patterns)
    print("Final patterns are ")
    print(patterns)
    recall = 0
    extraResults = 0
    for (pageLocation, seed) in testPageLocationAndSeed:
        pageContent = preprocessDocument(readPage(pageLocation))
        resultsPerPage = preprocessResults(list(extractSet(patterns, pageContent)))
        resultsPerPage = removeJunk(resultsPerPage)
        flag=True
        for s in seed:
            if not s in resultsPerPage:
                print(pageLocation)
                print("Following: ")
                print(s)
                flag=False
        if flag==True:
            recall+=1
#         else:
#             print("Seed results were")
#             print(seed)
#             print("resultsperpage is ")
#             print(resultsPerPage)
        extraElements = len(resultsPerPage) - len(seed)
        if extraElements>1:
            seedsLocation = os.path.dirname(pageLocation) + "/seed"
            allSeeds = readAllRelationSeeds(seedsLocation)
            totalCount = len(resultsPerPage)
            allSeedsCount = len(list(allSeeds.intersection(set(resultsPerPage))))
            extracount = totalCount - allSeedsCount
            extraResults += extracount
            print(pageLocation)
            print("Extra elements are ")
            print(set(resultsPerPage)-seed)
    seedsMissed = len(testPageLocationAndSeed)-recall
    return (patterns, seedsMissed, extraResults, len(testDirs))
    
# getAllPatternsByPlainStringMatch("tablesData/flipkart/corpus")








def getAllPatternsByRetainingOnlyClassValue(corpusLocation):
    directories             = getAllDirectoriesInLocation(corpusLocation)
    (trainDirs, testDirs)   = trainTestSplit(directories)
    pageLocationsAndSeed    = getPageLocationAndSeed(trainDirs)
    testPageLocationAndSeed = getPageLocationAndSeed(testDirs)
    specialWordsString      = getAllSpecialWordsAsString(pageLocationsAndSeed)
    (regExpIn, regExpOut)   = createRegExpFromWordStr(specialWordsString)
    leftContexts    = []
    betweenContexts = []
    rightContexts   = []
    for (pageLocation, seed) in pageLocationsAndSeed:
        pageContent = doProcessingWithoutClass(preprocessDocument(readPage(pageLocation)), regExpIn, regExpOut)
        (lcPerPage, bcPerPage, rcPerPage) = getContextsForAllSeeds(pageContent, seed)
#         (leftContextsPerPage, rightContextsPerPage) = getContextsForAllSeeds(pageContent, seed)
        leftContexts.extend(lcPerPage)
        betweenContexts.extend(bcPerPage)
        rightContexts.extend(rcPerPage)
    leftPatterns     = getLeftPatterns(leftContexts)
    betweenPatterns  = getBetweenPatterns(betweenContexts)
    rightPatterns    = getRightPatterns(rightContexts)
    patterns         = allPossibleTriples(leftPatterns, betweenPatterns, rightPatterns)
    print(patterns)
    patterns = filterPatterns(pageLocationsAndSeed, patterns, True, regExpIn, regExpOut)
    recall = 0
    extraResults = 0
    for (pageLocation, seed) in testPageLocationAndSeed:
        pageContent = doProcessingWithoutClass(preprocessDocument(readPage(pageLocation)), regExpIn, regExpOut)
        resultsPerPage = preprocessResults(list(extractSet(patterns, pageContent)))
        resultsPerPage = removeJunk(resultsPerPage)
        flag=True
        for s in seed:
            if not s in resultsPerPage:
                print(pageLocation)
                print("Following: ")
                print(s)
                flag=False
        if flag==True:
            recall+=1
#         else:
#             print("Seed results were")
#             print(seed)
#             print("resultsperpage is ")
#             print(resultsPerPage)
        extraElements = len(resultsPerPage) - len(seed)
        if extraElements>1:
            seedsLocation = os.path.dirname(pageLocation) + "/seed"
            allSeeds = readAllRelationSeeds(seedsLocation)
            totalCount = len(resultsPerPage)
            allSeedsCount = len(list(allSeeds.intersection(set(resultsPerPage))))
            extracount = totalCount - allSeedsCount
            extraResults += extracount
            print(pageLocation)
            print("Extra elements are ")
            print(set(resultsPerPage)-seed)
    seedsMissed = len(testPageLocationAndSeed)-recall
    return (patterns, seedsMissed, extraResults, len(testDirs), specialWordsString)
# getAllPatternsByRetainingOnlyClassValue("tablesData/amazon")


# getAllPatternsByRetainingOnlyClassValue("tablesData/flipkart/corpus")

def getPatternStore(eCommerceDataSetLocation):
    allWebsites = getAllDirectoriesInLocation(eCommerceDataSetLocation)
#     print("All websites are " + str(allWebsites))
    patternsStore = []
    for website in allWebsites:
        corpusLocation = website + "/corpus"
        websiteName = os.path.basename(website)
        (plainPatterns, plainMissed, plainJunkCount, plainTotal)           = getAllPatternsByPlainStringMatch(corpusLocation)
        (noValPatterns, noValMissed, noValJunkCount, noValTotal, wordsStr) = getAllPatternsByRetainingOnlyClassValue(corpusLocation)
#         print(website)
#         print("Plain missed is " + str(plainMissed))
#         print("junk missed is " + str(noValMissed))
        if plainMissed < noValMissed:
            patterns = (websiteName, "plainPattern", plainPatterns, plainMissed, plainJunkCount, plainTotal)
#             print(plainPatterns)
        elif plainMissed > noValMissed:
            patterns = (websiteName, "noValuePattern_"+wordsStr, noValPatterns, noValMissed, noValJunkCount, noValTotal)
        else:
            if plainJunkCount<=noValJunkCount:
#                 print("Here")
                patterns = (websiteName, "plainPattern", plainPatterns, plainMissed, plainJunkCount, plainTotal)
            else:
#                 print("There")
                patterns = (websiteName, "noValuePattern_"+wordsStr, noValPatterns, noValMissed, noValJunkCount, noValTotal)
#         if plainMissed < noValMissed:
#             patterns = (websiteName, "plainPattern", plainPatterns, plainMissed, plainJunkCount, plainTotal)
# #             print(plainPatterns)
#         else:
#             patterns = (websiteName, "noValuePattern_"+wordsStr, noValPatterns, noValMissed, noValJunkCount, noValTotal)
        patternsStore.append(patterns)
#     for p in patternsStore:
#         print(p)
    return patternsStore




def writeListToFile(loc, l):
    with open(loc, 'w') as f:
        for item in l:
            f.write(item+"\n")
            
            
            
            
# getPatternStore("tablesData")

def writeRelationPatternsToFile(fileLocation, productPatterns):
    header = "GroupId\tWebsiteName\tPatternType\tMissed\tJunk\tOutof\tLeftPattern\tMiddlePattern\tRightPattern"
    groupId = 1
    output = []
    output.append(header)
    for (website, pType, patterns, missed, junk, total) in productPatterns:
        rowPrefix = str(groupId) + "\t" + website + "\t" + pType
        rowPrefix+="\t" + str(missed) + "\t" + str(junk) + "\t" + str(total)
        for (l, m, r) in patterns:
            row = rowPrefix + "\t" + l + "\t" + m + "\t" + r
            output.append(row)
        groupId+=1
#     print("output is ")
#     for item in output:
#         print(item)
    writeListToFile(fileLocation, output)
    print("Output written at location: " + fileLocation)

In [31]:
def isDifferent(ch):
    if ch>='a' and ch<='z':
        return False
    elif ch>='A' and ch<='Z':
        return False
    elif ch>='0' and ch<='9':
        return False
    else:
        return True
def findExactWordLoc(document, seed):
    output = []
    for m in re.finditer(re.escape(seed), document):
        start = m.start()
        end   = m.end()
        output.append((start, end))
    print(output)
    for (s, e) in output:
        if s==0:
            l = ' '
            if e == len(document):
                r = ' '
            else:
                r = document[e]
        else:
            l = document[s-1]
            if e==len(document):
                r = ' '
            else:
                r = document[e]
#         print((s, e))
#         print(l)
#         print(r)
#         print()
#         print(str(l) + " " + str(isDifferent(l)))
#         print(sisDifferent(r))
        if isDifferent(l) and isDifferent(r):
#             print("l was " + l)
#             print("r was " + r)
            return s
    if len(output)==0:
        return -1
    else:
        (s, e) = output[0]
        return s
        
                
    
    
    
# findExactWordLoc("hi my na1me is sanj1k 1 kumar", "1")

s ="</div><ul class=\"_3dG3ix col col-9-12\"><li class=\"sNqDog\">1</li></ul></li><li class=\"_1KuY3T row\"><div class=\"vmXPri col col-3-12\">"
print(s[findExactWordLoc(s, "1"):])
# print(s[35:])
# print(s[58:])
# print(s[86:])

[(35, 36), (58, 59), (86, 87), (127, 128)]
1</li></ul></li><li class="_1KuY3T row"><div class="vmXPri col col-3-12">


In [99]:
print("Enter e-commerce data location: ")
eCommerceDataLocation = raw_input()
print("Enter outputLocation")
outputLocation = raw_input()
outputLocation+="/tablePatterns.tsv"
print("Patterns will be found at " + str(outputLocation))
patternsStore = getPatternStore(eCommerceDataLocation)
print("Pattern store is ")
print(patternsStore)
writeRelationPatternsToFile(outputLocation, patternsStore)
#categoryData
#featureSpecificationData

Enter e-commerce data location: 
tablesData
Enter outputLocation
newPatternsLearnt
Patterns will be found at newPatternsLearnt/tablePatterns.tsv
Left Patterns
['">']
Between
['</div><ul class="_3dG3ix col col-9-12" data-reactid="320"><li class="sNqDog" data-reactid="321">', 'Type: Narrow', '</div><ul class="_3dG3ix col col-9-12" data-reactid="389"><li class="sNqDog" data-reactid="390">', '</div><ul class="_3dG3ix col col-9-12" data-reactid="381"><li class="sNqDog" data-reactid="382">', '</div><ul class="_3dG3ix col col-9-12" data-reactid="452"><li class="sNqDog" data-reactid="453">', '</div><ul class="_3dG3ix col col-9-12" data-reactid="385"><li class="sNqDog" data-reactid="386">', 'Men&#39;s Black Trousers"></div></div></div><!-- react-empty: 5599 --></a><div class=""><a title="Arrow Sport', '+Men%27s+Black+Trousers_TROEVXR3DHBFFSFM_gamificationAndPersonalisation/recentlyViewed_3&cid=TROEVXR3DHBFFSFM">Arrow Sport', 'Men&#39;s Black Trousers"></div></div></div><!-- react-empty: 5861 --

This guy
('Type', 'Sports Jacket')
*************Removing*****************
('<div class="vmXPri col col-3-12">Type', 'Sports Jacket')
This guy
('Reversible', 'No')
*************Removing*****************
('<div class="vmXPri col col-3-12">Reversible', 'No')
This guy
('Color', 'Blue, Dark Blue')
*************Removing*****************
('<div class="vmXPri col col-3-12">Color', 'Blue, Dark Blue')
This guy
('Pattern', 'Printed')
*************Removing*****************
('<div class="vmXPri col col-3-12">Pattern', 'Printed')
This guy
('Pockets', '2 Pockets at Front')
*************Removing*****************
('<div class="vmXPri col col-3-12">Pockets', '2 Pockets at Front')
************Saving*******************
('Style Code', '83063302new navy')
************Saving*******************
('Hooded', 'Yes')
************Saving*******************
('Closure', 'Zipper')
returning
set([('Color', 'Blue, Dark Blue'), ('Pockets', '2 Pockets at Front'), ('Fabric Care', 'Machine Wash as per Tag'), ('Pattern', 'Pri

('<ul><li class=_1KuY3T row><div class=vmXPri col col-3-12>Model Name', 'DARBAR-PANAL')
This guy
('<li class=_1KuY3T row><div class=vmXPri col col-3-12>Model Number', '2DARBAR-2')
*************Removing*****************
('</li><li class=_1KuY3T row><div class=vmXPri col col-3-12>Model Number', '2DARBAR-2')
This guy
('<div class=vmXPri col col-3-12>Model Number', '2DARBAR-2')
*************Removing*****************
('<li class=_1KuY3T row><div class=vmXPri col col-3-12>Model Number', '2DARBAR-2')
This guy
('<div class=vmXPri col col-3-12>Model Name', 'DARBAR-PANAL')
*************Removing*****************
('<div><div class=_2Kp3n6><ul><li class=_1KuY3T row><div class=vmXPri col col-3-12>Model Name', 'DARBAR-PANAL')
This guy
('<div class=vmXPri col col-3-12>Model Name', 'DARBAR-PANAL')
*************Removing*****************
('<li class=_1KuY3T row><div class=vmXPri col col-3-12>Model Name', 'DARBAR-PANAL')
This guy
('Model Number', '2DARBAR-2')
*************Removing*****************
('<div 

[]
returning
set([])
returning
set([])
returning
set([])
returning
set([])
Left Patterns
['><td class="label">']
Between
['</td><td class="value">', '</span> </td> <td class="comparison_baseitem_column"> <span class="a-size-base a-color-base">', ':', '</span> </td> <td class="comparison_baseitem_column" style="background-color: rgb(255, 255, 222);"> <span class="a-size-base a-color-base">', 'Number</span> </td> <td class="comparison_baseitem_column" style="background-color: rgb(255, 255, 222);"> <span class="a-size-base a-color-base">']
Right
['</td></tr> <tr']
[('><td class="label">', '</td><td class="value">', '</td></tr> <tr'), ('><td class="label">', '</span> </td> <td class="comparison_baseitem_column"> <span class="a-size-base a-color-base">', '</td></tr> <tr'), ('><td class="label">', ':', '</td></tr> <tr'), ('><td class="label">', '</span> </td> <td class="comparison_baseitem_column" style="background-color: rgb(255, 255, 222);"> <span class="a-size-base a-color-base">', '</td>

[]
returning
set([])
tablesData/amazon/corpus/page9/page.html
Following: 
('Color', 'Maroon')
tablesData/amazon/corpus/page9/page.html
Following: 
('Upholstery material', 'Premium Fabric')
tablesData/amazon/corpus/page9/page.html
Following: 
('Item Shape', 'Rectangular')
tablesData/amazon/corpus/page9/page.html
Following: 
('Product Dimensions', '198 x 152 x 15 cm')
tablesData/amazon/corpus/page9/page.html
Following: 
('Item Part Number', 'SBS06')
tablesData/amazon/corpus/page9/page.html
Following: 
('Is Assembly Required', 'No')
tablesData/amazon/corpus/page9/page.html
Following: 
('Mattress Thickness', '6 Inches')
tablesData/amazon/corpus/page9/page.html
Following: 
('Number of Pieces', '1')
tablesData/amazon/corpus/page9/page.html
Following: 
('Item Weight', '29 Kg')
tablesData/amazon/corpus/page9/page.html
Following: 
('Item Model Number', 'SBS06')
tablesData/amazon/corpus/page9/page.html
Following: 
('Primary material', 'Spring')
tablesData/amazon/corpus/page9/page.html
Following:

In [84]:
l = "\">\"esab-ezis-a 5naps-a\"=ssalc ht< >rt<>ydobt< >\"inim-gnicaps-a eulavyek-a\"=ssalc \"1_noitces_snoitaci"
print(l[::-1])

ications_section_1" class="a-keyvalue a-spacing-mini"> <tbody><tr> <th class="a-span5 a-size-base">"


In [78]:
l1 = ["'>\"lebal\"=ssalc dt<>rt< >ydobt< >\"0\"=redrob \"0\"=gniddapllec \"0\"=gnicapsllec elbat< >\";kcolb:yalpsid\"='"]
l2 = ["'>\"yradnoces-roloc-a esab-ezis-a\"=ssalc naps< >\"loc_tsrif_elbat_nosirapmoc nmuloc_eman_etubirtta_nosi'", "'>\"lebal\"=ssalc dt<>rt< >rt/<>dt/<senohpdaeH>\"eulav\"=ssalc dt<>dt/<sliateD oiduA>\"lebal\"=ssalc dt<>rt'"]
print(l1)
print(l2)
l = [l1, l2]
getLeftPatterns(l)

['\'>"lebal"=ssalc dt<>rt< >ydobt< >"0"=redrob "0"=gniddapllec "0"=gnicapsllec elbat< >";kcolb:yalpsid"=\'']
['\'>"yradnoces-roloc-a esab-ezis-a"=ssalc naps< >"loc_tsrif_elbat_nosirapmoc nmuloc_eman_etubirtta_nosi\'', '\'>"lebal"=ssalc dt<>rt< >rt/<>dt/<senohpdaeH>"eulav"=ssalc dt<>dt/<sliateD oiduA>"lebal"=ssalc dt<>rt\'']


['> <tr><td class="label">\'']

In [75]:
def prefixStatus((l1, r1), (l2, r2)):
#     print(l1, r1)
#     print(l2, r2)
    if (l1.startswith(l2) or l1.endswith(l2)) and len(l1)>len(l2):
        return True
    else:
        if (r1.startswith(r2) or r2.startswith(r1)) and len(r1)>len(r2):
            return True
        else:
            return False
def removeJunk(s):
    l = list(s)
    total = len(l)
    output = []
    for index in range(0, total):
        flag=True
        for j in range(0, total):
            if index==j:
                continue
            if prefixStatus(l[index], l[j]):
                flag=False
        if flag==True:
            output.append(l[index])
    return output
                
removeJunk([("sanjeev", "kumar"), ("sanjeevika", "kumari")])
# print("sanjeevika".endswith("sanjeevika"))

[('sanjeev', 'kumar')]